In [1]:
import os
import cv2 as cv
import numpy as np
import pandas as pd
import joblib
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
c = 0
for files in os.listdir('./train'):
    c += 1
print(c)

25000


In [3]:
txt = "cat.0.jpg"

x = txt.split(".")

print(x)

['cat', '0', 'jpg']


In [25]:
data = []
for files in os.listdir('./train'):
    image = cv.imread(os.path.join('./train', files))
    image_resize = cv.resize(image,(100,100))
    gray = cv.cvtColor(image_resize, cv.COLOR_RGB2GRAY)
    label = files.split('.')[0]
    if label == 'cat':
        data.append([gray, np.array([0])])
    if label == 'dog':
        data.append([gray, np.array([1])])    

In [26]:
print(data[0:3])

[[array([[146, 152, 155, ..., 189, 185, 183],
       [146, 152, 155, ..., 192, 187, 184],
       [146, 152, 155, ..., 196, 188, 186],
       ...,
       [108, 110, 113, ...,   3,   3,   1],
       [108, 109, 112, ...,   2,   2,   2],
       [104, 106, 110, ...,   1,   1,   1]], dtype=uint8), array([0])], [array([[ 43,  45,  45, ...,  95, 179, 186],
       [ 40,  38,  40, ..., 105, 173, 187],
       [ 43,  38,  42, ...,  93, 170, 189],
       ...,
       [ 21,  20,  20, ...,  81,  70,  38],
       [ 24,  18,  21, ...,  55,  35,  26],
       [ 29,  20,  24, ...,  85,  15,  32]], dtype=uint8), array([0])], [array([[ 41,  28,  46, ..., 157, 158, 154],
       [ 45,  55,  61, ..., 157, 160, 156],
       [ 54,  59,  61, ..., 159, 162, 158],
       ...,
       [145, 146, 145, ..., 133, 136, 125],
       [159, 151, 142, ..., 134, 137, 125],
       [132, 115, 123, ..., 134, 136, 122]], dtype=uint8), array([0])]]


In [27]:
#the best practice is to shuffel the data so that the model would be robust
import random

random.shuffle(data)

In [28]:
x = []
y = []

for features,labels in data:
    x.append(features)
    y.append(labels)

In [ ]:
print(type(x))
print(x.shape)

In [30]:
x = np.array(x)
y = np.array(y)

#It would be much more easy for the computation if the values of the features are in the range between 0 and 1
x = x/255

In [31]:
print(type(x))
print(len(x))
print(x.shape)
print(y)

In [ ]:
print(type(y))

In [33]:
x = x.reshape(-1, 100, 100, 1)

In [34]:
print(x.shape)

(25000, 100, 100, 1)


In [35]:
'''
Resizing the data and training the model on huge datasets can be time consuming and computation expensive
So, the best way to use the model again is to save the training weights and to reuse the model without
actually training the model again.

There are many libraries to save files
I used 'joblib' as it is efficient in handling arrays
'''

joblib.dump(x, 'features.pkl')
joblib.dump(y, 'labels.pkl')

'''
To open the pickel files again use the load function

variable = joblib.load('filename.format')
'''

"\nTo open the pickel files again use the load function\n\nvariable = joblib.load('filename.format')\n"

In [ ]:
'''
Now that the data is prepared and sorted, we need to create the model with desired number of convolution
layers, pooling layers, and fully connected layers with desired number of filters and their sizes
'''

from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

model.add(Conv2D(64, (3,3), activation = 'relu', input_shape = x.shape[1:]))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3,3), activation = 'relu'))

model.add(Flatten())
model.add(Dense(128, input_shape = x.shape[1:], activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

model.summary()

In [37]:
'''
We need to define the optimisers and loss functions
optimisers are primarly use to define the learning rate for indivudual parameters
Loss function is the actual difference of the actual label and predicted label which has to be minimized
'''

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')


In [38]:
model.fit(x ,y, epochs = 3, validation_split = 0.1)

Epoch 1/3
704/704 [==============================] - 320s 454ms/step - loss: 0.6627 - accuracy: 0.5896 - val_loss: 0.5803 - val_accuracy: 0.6932
Epoch 2/3
704/704 [==============================] - 310s 441ms/step - loss: 0.5276 - accuracy: 0.7362 - val_loss: 0.4818 - val_accuracy: 0.7728
Epoch 3/3
704/704 [==============================] - 313s 444ms/step - loss: 0.4399 - accuracy: 0.7953 - val_loss: 0.4172 - val_accuracy: 0.8040


In [ ]:
model.save('trained')